# Exploring Twitter Data with Vector Databases and RAG Systems

This tutorial introduces the creation of a vector database and the use of a retrieval-augmented generation (RAG) system to explore Twitter data interactively. 

Please check [LBSocial](www.lbsocial.net)  on how to collect Twitter data. 

## Set up a Database and API Keys

Create a [MongoDB](www.mongodb.com) cluster and store the connection string in a safe place, such as AWS Secrets Manager. 
- key name: `connection_string`
- key value: <`the connection string`>, you need to type the password
- secret name: `mongodb`


You also need to purchase and your [oepnai](https://openai.com/) api key in AWS Secrets Manager:
- key name: `api_key`
- key value: <`your openai api key`>
- secret name: `openai`

## Install Python Libraries

- pymongo: manage the MongoDB database
- openai: create embeddings and resonpses 

In [1]:
pip install pymongo openai -q

Note: you may need to restart the kernel to use updated packages.


## Secrets Manager Function

In [2]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Import Python Libraries and Credentials

In [3]:
import pymongo
from pymongo import MongoClient
import json
import re
import os
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)


mongodb_connect = get_secret('mongodb')['connection_string']


## Connect to the MongoDB cluster

In [4]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection

## Utility Funcitons

- the `clean_tweet` function removes URLs in tweets
- the `get_embedding` function use openai to create tweet embeddings
- the `vector_search` function return relevent tweets based on a query

In [5]:
def clean_tweet(text):
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_pattern, '', text)

In [6]:
embedding_model= 'text-embedding-3-small'

def get_embedding(text):

    try:
        embedding = client.embeddings.create(input=text, model=embedding_model).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

In [7]:
def vector_search(query):

    query_embedding = get_embedding(query)
    if query_embedding is None:
        return "Invalid query or embedding generation failed."
    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "tweet_vector",
                "queryVector": query_embedding,
                "path": "tweet.embedding",
                "numCandidates": 1000,  # Number of candidate matches to consider
                "limit": 10  # Return top 10 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "tweet.text": 1 # return tweet text

            }
        }
    ]

    results = tweet_collection.aggregate(pipeline)
    return list(results)

## Tweets Embedding 

For more about text embeddings please read [Introducing text and code embeddings](https://openai.com/index/introducing-text-and-code-embeddings/)

In [8]:
from tqdm.auto import tqdm
tweets = tweet_collection.find()

for tweet in tqdm(list(tweets)):
    try:
        tweet_embedding = get_embedding(clean_tweet(tweet['tweet']['text']))
    #     print(tweet_embedding)

        tweet_collection.update_one(
            {'tweet.id':tweet['tweet']['id']},
            {"$set":{'tweet.embedding':tweet_embedding}}
        )
    except:
        print(f"""error in embedding tweet {tweet['tweet']['id']}""")
        pass


  0%|          | 0/100 [00:00<?, ?it/s]

## Create a Vector Index

For more about the MognoDB Vector database, please read [What are Vector Databases?](https://www.mongodb.com/resources/basics/databases/vector-databases)
This code creates a vector index following the [MongoDB official document](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#std-label-avs-types-vector-search).

In [9]:
# Create your index model, then create the search index

from pymongo.operations import SearchIndexModel
import time

search_index_model = SearchIndexModel(
  definition={
  "fields": [
    {
      "type": "vector",
      "path": "tweet.embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    }
  ]
},
  name="tweet_vector",
  type="vectorSearch"

)
result = tweet_collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")
# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None

if predicate is None:
  predicate = lambda index: index.get("queryable") is True

while True:
  indices = list(tweet_collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):

    break
  time.sleep(5)

print(result + " is ready for querying.")

New search index named tweet_vector is building.
Polling to check if the index is ready. This may take up to a minute.
tweet_vector is ready for querying.


In [11]:
user_query = 'gloucester'

for tweet in vector_search(user_query):
    print(tweet['tweet']['text'])

RT @rugger_red: Gloucester Rugby https://t.co/WYY6IvhamZ
RT @rugger_red: Gloucester Rugby https://t.co/WYY6IvhamZ
RT @rugger_red: Gloucester Rugby https://t.co/WYY6IvhamZ
Gloucester Catholic vs Palmyra | New Jersey High School Football
🔴 Live HD ▷ https://t.co/rdIaLLnFAZ
The Palmyra (NJ) varsity football team has a home non-conference game vs. Gloucester Catholic (Gloucester City, NJ) today @ 5p.
@eyed_loon There is a nice Wetherspoons at Gloucester docks too by the way. Plenty of other eateries too. Let us know if you go there.
I would have asked if I hadn’t misheard! Although Freddie plays for Gloucester so it would not have really been relevant https://t.co/oOte6BlVO0
NEXT UP! Back to league action as we take on Gloucester City at home. Adult tickets available on Fanbase at the following link. Concessions can be purchased on the turnstile, cash or card. @GCAFCofficial 

https://t.co/p9x1CAwGts
Gloucester Food Pantry Mobilizes for SNAP Shutdown – The Open Door Executive Director Juli

## Retrieval-Augmented Generation (RAG) 

For more about RAG, please read [Retrieval-Augmented Generation (RAG) with Atlas Vector Search](https://www.mongodb.com/docs/atlas/atlas-vector-search/rag/#std-label-avs-rag).

In [12]:
from openai import OpenAI

delimiter = '###'
chat_model = 'gpt-4o'
temperature = 0

chat_history = [{"role": "system", "content": """ you are a chatbot, answer user questions based on the returned tweets."""}]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})
    
    tweets = vector_search(prompt)
    chat_history.append({"role": "system", "content": f" here the returned tweets delimited by {delimiter}{tweets}{delimiter}"})

    response = client.chat.completions.create(
        model=chat_model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  study gloucester


Chatbot: The tweets mention several aspects of Gloucester:

1. **Sports**: Gloucester is referenced in the context of rugby and high school American football. Gloucester Rugby appears to have a following, as evidenced by multiple retweets, and there's a specific high school football game involving Gloucester Catholic mentioned.

2. **Dining and Attractions**: There is a mention of a Wetherspoons at Gloucester docks, along with other eateries, suggesting it's a place of interest for visitors and locals alike.

3. **Career Opportunities**: There's an opportunity for a career in governance through an apprenticeship at GlosCityHomes, highlighting potential employment or educational opportunities in the city.

4. **Transportation**: There are updates about train lines reopening between Lydney and Gloucester, which indicates active rail transportation services.

5. **Local Dialects**: A tweet discusses regional dialects possibly influenced by immigrant populations stretching from Plymouth to

You:  study glouvester rage


Chatbot: The search results for "Gloucester rage" did not return specific information related to that exact term. However, here are some insights from the tweets about Gloucester:

1. **Sports and Events**: Gloucester Rugby is mentioned repeatedly, suggesting it might be a significant part of the local culture. There's also a high school football game involving Gloucester Catholic.

2. **Local Services and Activities**: There are eateries at Gloucester docks, highlighting places locals or visitors might frequent.

3. **Traffic Update**: A traffic update mentions a crash on US-17 in Gloucester County, but no lanes were closed.

4. **Community Support**: The Gloucester Food Pantry is mobilizing to address challenges due to a SNAP shutdown, indicating active community support systems.

These aspects reflect community life and activities in Gloucester but do not specifically mention any "rage" instances or contexts. If you have a different focus, please clarify for more targeted informatio

You:  tell me about the darker side of gloucester


Chatbot: The returned tweets do not specifically highlight the darker side of Gloucester. However, there are a few hints that might reflect some challenges:

1. **Community Support for Food Security**: The Gloucester Food Pantry is mobilizing in response to a SNAP shutdown. This suggests there may be food security issues affecting parts of the Gloucester community.

2. **Political Discussion**: There are references to political discussions and issues being addressed in Gloucester County, indicating active engagement with potentially challenging topics facing the community.

Unfortunately, these tweets do not provide specific details about crime or other "dark" aspects usually associated with this type of inquiry. You might want to explore news articles or local community reports for more insight into issues such as crime, economy, or social challenges in Gloucester.


You:  quit 


Chatbot: If you were looking for information about quitting a job or any other topic related to "quit," the provided tweets don't contain relevant information. The tweets mostly discuss:

1. **Traffic and Transportation**: There is a mention of a crash in Gloucester County but with no lane closures, and rail lines reopening after an inspection.

2. **Community Support**: The Gloucester Food Pantry is addressing issues due to the SNAP shutdown.

3. **Sports and Events**: Gloucester Rugby and a local football match are highlighted.

4. **Miscellaneous**: There are mentions of vape and weed delivery services in multiple locations, including Gloucester.

If you have another specific topic or question in mind, feel free to ask!


## Reference

- *“Introducing Text and Code Embeddings.”* n.d. OpenAI. Accessed October 31, 2024. https://openai.com/index/introducing-text-and-code-embeddings/.
- *“What Are Vector Databases?”* n.d. MongoDB. Accessed October 31, 2024. https://www.mongodb.com/resources/basics/databases/vector-databases.
